In [1]:
import spacy
import pandas as pd

import json
import spacy
import requests
import random
import time

import spacy
import tqdm
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim

In [2]:
df = pd.read_csv("C:/Users/Rebeca OdC/Documents/A4/Social media analysis/obama_second_term.csv")
#corpus_text = df['lead_paragraph'].values

In [3]:
df['full_text']=df['headline']+df['lead_paragraph']
corpus_text = df['full_text'].values
df=df.dropna(subset=['full_text'])
df['lenght']=df['full_text'].apply(len)

In [4]:
#CONVERTING INTO LIST OF TOKEMS AND LEMATIZATION
nlp = spacy.load("en_core_web_sm")
corpus_text=df.full_text.values
documents_nlp = []
for d in tqdm.tqdm(nlp.pipe(corpus_text,n_process=5),total=len(corpus_text)):
    documents_nlp.append(d)
docs = [[token.lemma_.lower() for token in doc if  token.is_alpha] for doc in documents_nlp]

#To avoid 'or' and 'of', I remove words that have 2 characters:
docs = [[token for token in doc if len(token) > 2] for doc in docs]

#CREATING DICTIONNARY
id2word = Dictionary(docs)
id2word.filter_extremes(no_below=20, no_above=0.1)
corpus = [id2word.doc2bow(text) for text in docs]
#testing dictionnary
print(corpus[100])

100%|██████████| 1420/1420 [00:18<00:00, 74.97it/s] 


[(68, 1), (148, 1), (312, 1), (377, 1)]


In [5]:
#Finally, the model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=15,
                   passes=20,
                   chunksize=2000)
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
cm.get_coherence()

#docs_2_topics will store the probability distribution of topics over documents.
docs_2_topics = lda_model[corpus]
docs_2_topics[24]

[(8, 0.9481466)]

In [6]:
label_dict = {i: ', '.join([token for token, score in lda_model.show_topic(i, topn=10)]) for i in range(0, lda_model.num_topics)}
for topic in label_dict:
    print (topic,':\t', label_dict[topic])

0 :	 iran, nuclear, japan, minister, prime, military, program, abe, deal, shinzo
1 :	 security, policy, who, national, foreign, military, offer, may, government, washington
2 :	 trade, more, deal, than, oil, decade, pacific, nation, cuba, nuclear
3 :	 clinton, hillary, vietnam, foreign, policy, into, their, who, america, republican
4 :	 russia, putin, war, taiwan, vladimir, cold, russian, how, other, after
5 :	 kerry, secretary, john, beijing, when, week, what, iran, sea, close
6 :	 south, sea, military, dispute, island, between, defense, tension, korea, missile
7 :	 climate, leader, meeting, change, relationship, between, two, jinping, take, first
8 :	 korea, north, nuclear, south, korean, test, missile, human, weapon, official
9 :	 transcript, morning, debate, news, your, presidential, service, federal, briefinggood, republican
10 :	 company, they, report, make, morning, after, agenda, their, other, take
11 :	 get, your, briefing, here, you, kong, sign, email, what, want
12 :	 big, t

In [7]:
pyLDAvis.enable_notebook()
visualisation = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)
visualisation

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.130728 -0.084675       1        1  8.285926
1     -0.005217  0.001176       2        1  7.073055
2     -0.045917 -0.125616       3        1  7.723443
3     -0.084578  0.042754       4        1  6.546133
4     -0.068098 -0.020002       5        1  6.981038
5      0.065216 -0.013607       6        1  4.943229
6      0.198346  0.075092       7        1  6.025028
7     -0.019271 -0.079919       8        1  9.694540
8      0.224404 -0.002683       9        1  8.101092
9     -0.001015  0.294761      10        1  3.563786
10    -0.095346 -0.018888      11        1  8.707293
11    -0.194703  0.109201      12        1  4.892382
12    -0.070951 -0.068624      13        1  5.894060
13     0.020034  0.022585      14        1  5.687449
14    -0.053630 -0.131554      15        1  5.881543, topic_info=        Term        Freq       Total Category  logprob  loglift
83     korea  262.000000  262.000000  Default  30.0000  30.0000
87     north  236.000000  236.000000  Default  29.0000  29.0000
202    south  206.000000  206.000000  Default  28.0000  28.0000
37   nuclear  213.000000  213.000000  Default  27.0000  27.0000
82      iran  170.000000  170.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
170     last   15.299362  117.430698  Topic15  -4.4034   0.7953
68      more   16.309590  142.386638  Topic15  -4.3394   0.6666
181     long   12.745484   85.298955  Topic15  -4.5860   0.9324
56    leader   13.466533  151.285506  Topic15  -4.5310   0.4144
127     take   12.177603  114.556912  Topic15  -4.6316   0.5919

[680 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
174       1  0.979512     abe
94        2  0.384929  abroad
94        4  0.336813  abroad
94        6  0.048116  abroad
94        8  0.096232  abroad
...     ...       ...     ...
341      14  0.022536    york
328       6  0.084025     you
328      12  0.882259     you
329      10  0.372837    your
329      12  0.612519    your

[2690 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [8]:
pyLDAvis.save_html(visualisation, 'obama_2_topic_modelling_visualisation.html')

In [9]:
topic_2_docs={}
for i, doc_2_topics in enumerate(docs_2_topics):
    for (topic,strength) in doc_2_topics:
        topic_2_docs.setdefault(topic,[]).append((i,strength))

In [10]:
#if we want to find the best article(s) for a topic
topic_number=1
best_doc = sorted(topic_2_docs[topic_number], key=lambda item: item[1], reverse=True)
print(label_dict[topic_number])
for doc in best_doc[:10]:
    print (corpus_text[doc[0]],doc[1])

security, policy, who, national, foreign, military, offer, may, government, washington
Michael Flynn, Anti-Islamist Ex-General, Offered Security Post, Trump Aide SaysWASHINGTON — President-elect Donald J. Trump has offered the post of national security adviser to Lt. Gen. Michael T. Flynn, potentially putting a retired intelligence officer who believes Islamist militancy poses an existential threat in one of the most powerful roles in shaping military and foreign policy, according to a top official on Mr. Trump’s transition team. 0.957575
Michael Flynn, Anti-Islamist Ex-General, Offered Security Post, Trump Aide SaysWASHINGTON — President-elect Donald J. Trump has offered the post of national security adviser to Lt. Gen. Michael T. Flynn, potentially putting a retired intelligence officer who believes Islamist militancy poses an existential threat in one of the most powerful roles in shaping military and foreign policy, according to a top official on Mr. Trump’s transition team. 0.9575